In [1]:
!pip install azure-storage-blob



[notice] A new release of pip available: 22.2.2 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os
import time
import json

# Ruta local al directorio de OneDrive
local_folder = os.path.expanduser("./capturas_camaras")

# Datos de conexión a Azure Blob Storage
connection_string = ""
container_name = "capturas"
checkpoint_file = "sync_checkpoint.json"
sync_interval_seconds = 3600  # cada 1 hora

# 🚀 Conexión a Azure Blob
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)


In [ ]:
# 📂 Cargar checkpoint
def load_checkpoint():
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            return json.load(f)
    return {}

# 💾 Guardar checkpoint
def save_checkpoint(data):
    with open(checkpoint_file, 'w') as f:
        json.dump(data, f)

# 🔄 Sincronizar carpeta
def sync_folder_to_blob(local_path, container_client, checkpoint):
    updated_checkpoint = checkpoint.copy()
    for root, dirs, files in os.walk(local_path):
        for file in files:
            file_path_on_disk = os.path.join(root, file)
            rel_path = os.path.relpath(file_path_on_disk, local_path).replace("\\", "/")
            last_modified = os.path.getmtime(file_path_on_disk)

            if rel_path in checkpoint and checkpoint[rel_path] >= last_modified:
                continue  # Ya sincronizado

            try:
                print(f"Subiendo: {rel_path}")
                with open(file_path_on_disk, "rb") as data:
                    container_client.upload_blob(
                        name=rel_path,
                        data=data,
                        overwrite=True
                    )
                updated_checkpoint[rel_path] = last_modified
            except Exception as e:
                print(f"Error subiendo {rel_path}: {e}")
    return updated_checkpoint

# ⏱️ Bucle principal
print("⏳ Iniciando sincronización cada hora...")
while True:
    try:
        checkpoint = load_checkpoint()
        new_checkpoint = sync_folder_to_blob(local_folder, container_client, checkpoint)
        save_checkpoint(new_checkpoint)
        print(f"✅ Sincronización completada. Esperando {sync_interval_seconds // 60} minutos...\n")
    except Exception as e:
        print(f"⚠️ Error en sincronización: {e}")
    
    time.sleep(sync_interval_seconds)


⏳ Iniciando sincronización cada hora...
Subiendo: registro_capturas.csv
Subiendo: camara_02605/captura_02605_20250421_213412.png
Subiendo: camara_02605/captura_02605_20250421_214105.png
Subiendo: camara_02605/captura_02605_20250421_214757.png
Subiendo: camara_02605/captura_02605_20250421_215449.png
Subiendo: camara_02605/captura_02605_20250421_220141.png
Subiendo: camara_02605/captura_02605_20250421_220833.png
Subiendo: camara_02605/captura_02605_20250421_221525.png
Subiendo: camara_02605/captura_02605_20250421_222216.png
Subiendo: camara_02605/captura_02605_20250421_222908.png
Subiendo: camara_02605/captura_02605_20250421_223559.png
Subiendo: camara_02605/captura_02605_20250421_224251.png
Subiendo: camara_02605/captura_02605_20250421_224943.png
Subiendo: camara_02605/captura_02605_20250421_225636.png
Subiendo: camara_02605/captura_02605_20250421_230328.png
Subiendo: camara_02605/captura_02605_20250421_231020.png
Subiendo: camara_02605/captura_02605_20250421_231713.png
Subiendo: camara

: 